# Basic Initialization 

In [32]:
import os
import cv2
import numpy as np

import imagehash
from PIL import Image
from rembg import remove
import imghdr
import dlib

from tqdm import tqdm

import io
import time
from shutil import move
import io
from IPython.display import clear_output

from tqdm import tqdm

In [33]:
folder_loc = r"C:\Users\parvs\Downloads\amber"

# Format for below: Abbreviation_full_form(init value)
# Min_Confidence(0.9), Min_Size(0.009) (proportional to image size), Min_SharpeningStat(100), Blur_Threshold(100)
mc, ms, mss, bt = 0.9, 0.01, 100, 100

detector = dlib.get_frontal_face_detector()

## Basic Image Processing

### Convert all images to PNG for easier reading

In [26]:
def convert_images_to_png(source_folder):
    if not os.path.exists(source_folder):
        print(f"The folder {source_folder} does not exist.")
        return

    for filename in tqdm(os.listdir(source_folder), desc = "Processing images to .png"):
        if filename.lower().endswith(('.jpg', '.jpeg', '.bmp', '.gif', '.tiff', '.webp')):
            file_path = os.path.join(source_folder, filename)
            with Image.open(file_path) as img:
                new_filename = os.path.splitext(filename)[0] + '.png'
                new_file_path = os.path.join(source_folder, new_filename)
                img.save(new_file_path, 'PNG')
                os.remove(file_path)

convert_images_to_png(folder_loc)

Processing images to .png: 100%|██████████| 114/114 [00:00<00:00, 170.74it/s]


### Remove Same Images

In [27]:
def remove_black_bars(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)
        cropped_image = image[y:y+h, x:x+w]
        return cropped_image
    return image

def process_images(folder_path):
    duplicate_images_folder = os.path.join(folder_path, "Duplicate_Images")
    if not os.path.exists(duplicate_images_folder):
        os.makedirs(duplicate_images_folder)
    
    image_hashes = {}
    files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp'))]
    for filename in tqdm(files, desc="Processing Images"):
        file_path = os.path.join(folder_path, filename)
        image = cv2.imread(file_path)
        if image is None:
            continue
        
        image_no_black_bars = remove_black_bars(image)
        
        pil_image = Image.fromarray(cv2.cvtColor(image_no_black_bars, cv2.COLOR_BGR2RGB))
        hash = str(imagehash.average_hash(pil_image))
        
        # Check for duplicates
        if hash in image_hashes:
            print(f"Duplicate found: {filename} is a duplicate of {image_hashes[hash]}")
            os.rename(file_path, os.path.join(duplicate_images_folder, filename))
        else:
            image_hashes[hash] = filename

process_images(folder_loc)

Processing Images: 100%|██████████| 111/111 [00:03<00:00, 30.88it/s]


### Checks Images for Suitability Check (Face check, etc)

In [29]:
current_folder = os.getcwd()

modelFile = os.path.join(current_folder, "opencv_face_detector_uint8.pb")
configFile = os.path.join(current_folder, "opencv_face_detector.pbtxt")

if os.path.exists(modelFile) != True or os.path.exists(configFile) != True:
    print("ERROR: KEY FILES FOR RUNNING THIS MODEL NOT FOUND, PLEASE FIND THEM AND INSTALL THEM")

else:
    net = cv2.dnn.readNetFromTensorflow(modelFile, configFile)

    def calculate_image_sharpness(image):
        
        """
        Calculate the sharpness of an image using the variance of the Laplacian
        """
        
        if image is None or image.size == 0:
            return 0
        return cv2.Laplacian(image, cv2.CV_64F).var()

    def is_image_blurry(image, blur_threshold):
        
        """
        Check if an image is blurry using the variance of the Laplacian method.
        """
        
        variance_of_laplacian = cv2.Laplacian(image, cv2.CV_64F).var()
        return variance_of_laplacian < blur_threshold

    def detect_faces_and_evaluate(image, min_confidence, min_size, min_sharpness, blur_threshold):
        
        """
        Detect faces in an image and evaluate based on size, sharpness, and blurriness.
        """
        
        if image is None or image.size == 0:
            return []
        if is_image_blurry(image, blur_threshold):
            return []
        (h, w) = image.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        faces_detected = []
        for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > min_confidence:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                startX, startY, endX, endY = max(0, startX), max(0, startY), min(w, endX), min(h, endY)
                if startX >= endX or startY >= endY:
                    continue
                face_region = image[startY:endY, startX:endX]
                if face_region.size == 0:
                    continue
                face_size = (endX - startX) * (endY - startY)
                face_sharpness = calculate_image_sharpness(face_region)
                if face_size > min_size and face_sharpness > min_sharpness:
                    faces_detected.append((confidence, face_size, face_sharpness))
        return faces_detected

    def remove_black_bars(image):
        
        """
        Remove black bars from an image.
        """

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            x, y, w, h = cv2.boundingRect(largest_contour)
            cropped_image = image[y:y+h, x:x+w]
            return cropped_image
        return image

    def select_best_images(
            folder_path:str, 
            min_confidence:float = 0.9, 
            min_size:float = 0.009, 
            min_sharpness:float = 100, 
            blur_threshold:float = 100
            ):
        
        selected_images_folder = os.path.join(folder_path, "SelectedImages")
        if not os.path.exists(selected_images_folder):
            os.makedirs(selected_images_folder)
        
        files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp'))]
        for filename in tqdm(files, desc="Evaluating Images"):
            file_path = os.path.join(folder_path, filename)
            image = cv2.imread(file_path)
            if image is None:
                continue
            
            image_no_black_bars = remove_black_bars(image)
            
            faces_detected = detect_faces_and_evaluate(image_no_black_bars, min_confidence, min_size * image_no_black_bars.size, min_sharpness, blur_threshold)
            if len(faces_detected) == 1:
                move(file_path, os.path.join(selected_images_folder, filename))

    select_best_images(folder_loc, mc, ms, mss, bt)

    selected_folder = os.path.join(folder_loc, "SelectedImages")

Evaluating Images: 100%|██████████| 37/37 [00:05<00:00,  6.96it/s]


## Advanved Image Cropping

### Remove Background of Images

In [30]:
def remove_background_from_images(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    for filename in tqdm(image_files, desc="Processing Images"):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        with open(input_path, 'rb') as input_file:
            input_image = input_file.read()

            output_image = remove(input_image)

            output_image = Image.open(io.BytesIO(output_image))
            output_image.save(output_path)

remove_background_from_images(selected_folder, os.path.join(selected_folder, "No_BG"))

Processing Images:   0%|          | 0/74 [00:00<?, ?it/s]

*************** EP Error ***************
EP Error D:\a\_work\1\s\onnxruntime\python\onnxruntime_pybind_state.cc:456 onnxruntime::python::RegisterTensorRTPluginsAsCustomOps Please install TensorRT libraries as mentioned in the GPU requirements page, make sure they're in the PATH or LD_LIBRARY_PATH, and that your GPU is supported.
 when using ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
Falling back to ['CUDAExecutionProvider', 'CPUExecutionProvider'] and retrying.
****************************************


Processing Images:   1%|▏         | 1/74 [00:04<06:01,  4.95s/it]

*************** EP Error ***************
EP Error D:\a\_work\1\s\onnxruntime\python\onnxruntime_pybind_state.cc:456 onnxruntime::python::RegisterTensorRTPluginsAsCustomOps Please install TensorRT libraries as mentioned in the GPU requirements page, make sure they're in the PATH or LD_LIBRARY_PATH, and that your GPU is supported.
 when using ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
Falling back to ['CUDAExecutionProvider', 'CPUExecutionProvider'] and retrying.
****************************************


Processing Images:   3%|▎         | 2/74 [00:09<05:54,  4.92s/it]

...

Processing Images:  97%|█████████▋| 72/74 [06:14<00:09,  4.79s/it]

*************** EP Error ***************
EP Error D:\a\_work\1\s\onnxruntime\python\onnxruntime_pybind_state.cc:456 onnxruntime::python::RegisterTensorRTPluginsAsCustomOps Please install TensorRT libraries as mentioned in the GPU requirements page, make sure they're in the PATH or LD_LIBRARY_PATH, and that your GPU is supported.
 when using ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
Falling back to ['CUDAExecutionProvider', 'CPUExecutionProvider'] and retrying.
****************************************


Processing Images:  99%|█████████▊| 73/74 [06:21<00:05,  5.30s/it]

*************** EP Error ***************
EP Error D:\a\_work\1\s\onnxruntime\python\onnxruntime_pybind_state.cc:456 onnxruntime::python::RegisterTensorRTPluginsAsCustomOps Please install TensorRT libraries as mentioned in the GPU requirements page, make sure they're in the PATH or LD_LIBRARY_PATH, and that your GPU is supported.
 when using ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
Falling back to ['CUDAExecutionProvider', 'CPUExecutionProvider'] and retrying.
****************************************


Processing Images: 100%|██████████| 74/74 [06:26<00:00,  5.22s/it]


### Crop Image to Face

In [35]:
failed_img = []
face_failed = []

# def faceCrop(folder_dir, fName, img, imp=1, x=512, y=512):
#     cropped_folder_path = os.path.join(folder_dir, "Cropped_Images")
#     if not os.path.exists(cropped_folder_path):
#         os.makedirs(cropped_folder_path)

#     detector = dlib.get_frontal_face_detector()
#     _, ext = os.path.splitext(fName)
    
#     fName = fName.split(".")[0]
#     if ext.lower() not in ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp']:
#         print(f"Unsupported image format: {ext}")
#         failed_img.append(fName)
#         return

#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     faces = detector(gray)
#     if len(faces) == 0:
#         face_failed.append(fName)
#     else:
#         importance = imp
#         areas = [face.width() * face.height() for face in faces]
#         max_area_index = np.argmax(areas)
#         fx, fy, fw, fh = faces[max_area_index].left(), faces[max_area_index].top(), faces[max_area_index].width(), faces[max_area_index].height()

#         fx = max(0, fx - fw)
#         fy = max(0, fy - fh)
#         fw = min(img.shape[1] - fx, fw * 3)
#         fh = min(img.shape[0] - fy, fh * 3)

#         if fw > fh:
#             fx += (fw - fh) // 2
#             fw = fh
#         else:
#             fy += (fh - fw) // 2
#             fh = fw

#         cropped = img[fy:fy+fh, fx:fx+fw]
#         cropped = cv2.resize(cropped, (x, y))

#         cv2.imwrite(os.path.join(cropped_folder_path, fName + '_cropped' + '.png'), cropped)

def faceCrop(folder_dir, fName, img, imp=1, x=512, y=512):
    detector = dlib.get_frontal_face_detector()
    _, ext = os.path.splitext(fName)
    
    fName = fName.split(".")[0]
    if ext.lower() not in ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp']:
        print(f"Unsupported image format: {ext}")
        failed_img.append(fName)
        return

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    if len(faces) == 0:
        None
        # print("--> TO BE ADDED")
        face_failed.append(fName)
    else:
        importance = imp
        areas = [face.width() * face.height() for face in faces]
        max_area_index = np.argmax(areas)
        fx, fy, fw, fh = faces[max_area_index].left(), faces[max_area_index].top(), faces[max_area_index].width(), faces[max_area_index].height()

        # Increase the size of the bounding box to include more area around the face
        fx = max(0, fx - fw)
        fy = max(0, fy - fh)
        fw = min(img.shape[1] - fx, fw * 3)
        fh = min(img.shape[0] - fy, fh * 3)

        # Crop the largest possible square within the bounding box
        if fw > fh:
            fx += (fw - fh) // 2
            fw = fh
        else:
            fy += (fh - fw) // 2
            fh = fw

        cropped = img[fy:fy+fh, fx:fx+fw]

        # Resize the cropped image to the specified dimensions
        cropped = cv2.resize(cropped, (x, y))

        cv2.imwrite(os.path.join(folder_dir, fName + '_cropped' + '.png'), cropped)

def main_call(folder_path, x=512, y=512):
    resized_folder_path = os.path.join(folder_path, "Cropped_Images")
    
    if not os.path.exists(resized_folder_path):
        print(r"Made /resized folder")
        os.makedirs(resized_folder_path)

    if os.path.isdir(folder_path):
        print("Valid Folder Location")
        files = os.listdir(folder_path)
        print("Number of files: ", len(files))
        itr = 0
        for i in tqdm(files, desc="Processing Files"):
            itr += 1
            file_loc = os.path.join(folder_path, i)
            if os.path.isfile(file_loc) and imghdr.what(file_loc):
                # print("{}/{}".format(itr, len(files)), i, ": is image file")
                img = cv2.imread(file_loc)
                if img is None:
                    print(f"Failed to load image: {i}:", "Cause: Empty !")
                    failed_img.append(i)
                    continue  # Skip this file and continue with the next iteration
                faceCrop(resized_folder_path, i, img, 1, x, y)
            elif os.path.isdir(file_loc):
                print(i, ": Is a folder")
            else:
                print(i, ": Is not a supported Image File")

    # Clear the output
    clear_output(wait=True)

    time.sleep(5)

    print("DONE !!")

    time.sleep(3)

    if len(failed_img) != 0:
        print("\nThese images failed: \nReason: Invalid to load: \n", failed_img, "\nCount:", len(failed_img))

    if len(face_failed) != 0:
        print("\nThese images failed: \nReason: Face not found: \n", face_failed, "\nCount:", len(face_failed))

main_call(selected_folder, 512, 512)

DONE !!
